In [28]:
from Bio import Entrez
import redis
import json

In [55]:
# Downloading data with query: 'single+cell+rna'
Entrez.email = "yunfang@chanzuckerberg.com"
handle = Entrez.esearch(db="gds", retmax=500, retstart=5000, term="single cell rna")
record = Entrez.read(handle)
handle.close()

In [40]:
redis_store = redis.StrictRedis(host='localhost', port=6379, db=2)
SR_PREFIX = 'sr:pg:'
PAGE_SIZE = 500


In [ ]:
#redis_store.set('sdfdf','sdfdf')
import time

page = 111
page_size = PAGE_SIZE
while page_size >= PAGE_SIZE:
    print ("Downloading page %d" % page)
    handle = Entrez.esearch(db="gds", retmax=PAGE_SIZE, retstart=page*PAGE_SIZE, term="single cell rna")
    record = Entrez.read(handle)
    handle.close()
    json_str = json.dumps(record['IdList'])
    redis_key = "%s%d" %(SR_PREFIX, page)
    redis_store.set(redis_key, json_str)
    page_size = len(record['IdList'])
    page+=1
    time.sleep(2)



In [ ]:
# validate the data 
page = 0
while page <= 111:
    redis_key = "%s%d" %(SR_PREFIX, page)
    res = redis_store.get(redis_key)
    #print "key: %s len: %d" % (redis_key, len(res))
    page+=1
    


In [ ]:
def fetch_esummary(id_str):
    while True:
        try:
            handle = Entrez.esummary(db="gds", id=id_str)
            records = Entrez.read(handle)
            handle.close()
            
            for r in records: 
                key = "%s%s" %(GDS_PREFIX, r['Id'])
                val = json.dumps(r)
                redis_store.set(key, val)
            return
        except Exception:
            print("ERROR getting %s" % id_str)
            time.sleep(5)
    

page = 111
GDS_PREFIX = 'gds:'
while page <= 111:
    print("Processing page %d" % page)
    redis_key = "%s%d" %(SR_PREFIX, page) 
    res = redis_store.get(redis_key) 
    id_list = json.loads(res)
    id_str = id_list[0]
    for idx in range(1,len(id_list)):
        if idx % 5 == 0:
            print ("  idx: %d key: %s" % (idx, id_str))
            fetch_esummary(id_str)
            time.sleep(1)
            id_str = id_list[idx]
        else:
            id_str = id_str + ',' + id_list[idx]
    print ("  idx: %d key: %s" % (idx, id_str))
    fetch_esummary(id_str)
    page += 1
    

In [76]:
# VEGA (table. visualization)

idx: 499 key: 200085527,200084281,200089479,200081674,200081673


In [101]:
# randomly generated some data
import random
import pprint 
pp = pprint.PrettyPrinter(indent=4) 

page = 0

while page <= 111:
    redis_key = "%s%d" %(SR_PREFIX, page) 
    id_list = json.loads(redis_store.get(redis_key))
    selected = random.sample(id_list, 5)
    
    for doc_id in selected:
        redis_key = 'gds:' + doc_id
        rec = json.loads(redis_store.get(redis_key))
        print pp.pprint(rec)
        print "================================================"
    page +=1

#rec = json.loads(redis_store.get('gds:300004949'))
#rec

{   u'Accession': u'GDS1550',
    u'ExtRelations': [],
    u'FTPLink': u'ftp://ftp.ncbi.nlm.nih.gov/geo/datasets/GDS1nnn/GDS1550/',
    u'GDS': u'1550',
    u'GEO2R': u'',
    u'GPL': u'90',
    u'GSE': u'3200',
    u'Id': u'1550',
    u'Item': [],
    u'PDAT': u'2005/08/26',
    u'PlatformTaxa': u'Saccharomyces cerevisiae',
    u'PlatformTitle': u'[YG_S98] Affymetrix Yeast Genome S98 Array',
    u'Projects': [],
    u'PubMedIds': [16246724],
    u'Relations': [],
    u'SSInfo': u'time;time;time;time;time;time;agent;agent',
    u'Samples': [   {   u'Accession': u'GSM71925', u'Title': u'0 Hr. Timepoint'},
                    {   u'Accession': u'GSM71926', u'Title': u'2 Hr. Timepoint'},
                    {   u'Accession': u'GSM71927', u'Title': u'4 Hr. Timepoint'},
                    {   u'Accession': u'GSM71928', u'Title': u'5 Hr. Timepoint'},
                    {   u'Accession': u'GSM71929', u'Title': u'6 Hr. Timepoint'},
                    {   u'Accession': u'GSM71930', u'Title':

In [102]:
page = 0
nsamples = 0
npubmed_articles = 0
n_points = 0
while page <= 111:
    redis_key = "%s%d" %(SR_PREFIX, page) 
    id_list = json.loads(redis_store.get(redis_key)) 
    n_points += len(id_list)
    for doc_id in id_list:
        redis_key = 'gds:' + doc_id
        rec = json.loads(redis_store.get(redis_key))
        npubmed_articles += len(rec['PubMedIds'])
        nsamples += int(rec['n_samples'])
    page +=1

print ("# GSM samples: %d, # pubmed articles: %d, # data points: %d" %(nsamples, npubmed_articles, n_points))

# GSM samples: 710490, # pubmed articles: 5900, # data points: 55787
